## Implementation of CNN for Simpson Image Recognition

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from PIL import Image
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import os
import time


ModuleNotFoundError: No module named 'PIL'

In [ ]:
torch.manual_seed(42)  # for reproducibility

In [7]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2)

        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(32 * 56 * 56, 500)  # after two downsampling, the image size is (224/2/2) = 56  
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(500, num_figure)
    

    def forward(self, x):

        x = self.relu1(self.conv1(x))
        x = self.maxpool1(x)
    
        x = self.relu2(self.conv2(x))
        x = self.maxpool2(x)

        x = self.dropout(x)

        x = x.view(x.size(0), -1)  # flatten the tensor

        x = self.fc1(x)
        x = self.relu3(x)

        x = self.fc2(x)

        return x

In [8]:
# number of Simpson characters
num_figure = 20
model = ConvNet()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Set criterion
criterion = nn.CrossEntropyLoss()

# Using L2 regularization
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Setup scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Setup early stopping
stop_training = False
count_early_stop = 0
best_loss = None

# keep training until early stop condition met
while not stop_training:
  
  for phase in ['train', 'val']:
    if phase == 'train':
      model.train()
    else:
      model.eval()

    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders[phase]:
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      # forward
      with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)

        # backward + optimize only if in training phase
        if phase == 'train':
          loss.backward()
          optimizer.step()

      # update loss
      running_loss += loss.item() * inputs.size(0)
      running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    # check early stop conditions every epoch
    if phase == 'val':
      if best_loss == None or epoch_loss < best_loss:
        best_loss = epoch_loss
        count_early_stop = 0
      else:
        count_early_stop += 1
        if count_early_stop > 3:  # stop if validation loss does not improve for 3 epochs
          stop_training = True

  scheduler.step()

NameError: name 'dataloaders' is not defined